In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [5]:
def web_scraper(url):
    print('Running Engine')
    #url = input('Please Enter the url')
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    print(f'This is the Page Title \n{soup.title.text}')
    
    link = [str(i).split()[1].split('"')[1] for i in soup.select('h1')] # creating the Link list
    print('Link Collection Successful 100%')
    title = [i.text for i in soup.select('h1')]
    print('Title Collection Successful 100%')
    
    #Creating a dataframe of our dataset which contains title and link
    data = pd.DataFrame(data = {'Title':title, 'Link':link}).drop_duplicates().reset_index().drop('index',axis = 1)
    #Creating our Location
    data['Location'] = location(data) 
    data['Location'] = update_lc(data)
    data = data[data.Location != 'No Location'].reset_index().drop('index', axis = 1) 
    
    
    data['Year'] = year_update(data)[0] #Year column
    data['Year'] = year_clean(data)
    #data['Content'] = year_update(data)[1] #Content column
    print('Year Collection Successful 100%')
    #data = content(data)

    return data

In [6]:
def years(base):
    work = []
    year = []
    content = []
    if len(base) > 1:
        for i in base:
            if 'work' in str(base).lower() or 'experience' in str(base).lower() or 'minimum' in str(base).lower():
                work.append(i)
                content.append(i)
            else:
                #print(f'{base}')
                work.append('No Experience 0 years')
                content.append(base[0])
        
    else:
        if 'work' in str(base).lower() or 'experience' in str(base).lower() or 'minimum' in str(base).lower():
            work.append(base[0])
            content.append(base)
        else:
            #print(f'{base}')
            work.append('No Experience 0 years')
            content.append(base)
        
    for i in work[0]:
        for j in i :
            if str(j) in'0123456789':
                year.append(j)
    return year, content

In [7]:
def year_update(data):
    year = []
    content = []
    for index, i in enumerate(data['Link']):
        pagex = requests.get(i)
        soupx = BeautifulSoup(pagex.content, 'html.parser')
        x = [i for i in soupx.find_all('li') if 'year' in str(i)]
        #print(index)
        year.append(years(x)[0])
        content.append(years(x)[1])
    return year, content

In [8]:
def year_clean(data):
    dura = []
    for i in data.Year:
        if len(i) > 1:
            if int(i[1]) == 0:
                dura.append(str(i[0])+str(i[1]))
            else:
                dura.append(str(i[0]) +"-"+str(i[1]))
        else:
            if i != []:
                dura.append(i[0])
            else:
                dura.append(0)
    return dura

In [9]:
def content(data):
    edu = []
    for val in data['Content']:
        for i in val:
            for j in i:
                edu.append(j)
    me = []
    for val in edu[:len(data.Content)]:
        if '<li>' in str(val):
                me.append(str(val).split('>')[1].split('<')[0])
        else:
            me.append(val)
    data['Content'] = me
    return data

In [10]:
def location(data):
    locat = []
    for index, url in enumerate(data['Link']):
        pagey = requests.get(url)
        soupy = BeautifulSoup(pagey.content, 'html.parser')
        loc = [i for i in soupy.find_all('p') if 'location' in str(i).lower()]
        if loc != []:
            mee = [i for i in BeautifulSoup(str(loc[0])).get_text().split('\n') if 'location' in i.lower()][0].split('\r')[0].split(':')[-1]
            locat.append(mee)
        else:
            locat.append('No Location')
    return locat

In [11]:
def update_lc(data):
    lc = []
    for val in data['Location']:
        if 'abuja' in val.lower():
            lc.append('Abuja')
        elif 'lagos' in val.lower():
            lc.append('Lagos')
        else:
            lc.append('No Location')
    return lc

In [ ]:
data = web_scraper(url)

In [240]:
data_saver_name =[]
num = int(input('Enter the number:'))

col = ['Title', 'Link', 'Year', 'Location']
new_df = pd.DataFrame(columns=col)

for i in range(num):
    url = f'https://www.hotnigerianjobs.com/role/350/{i}/'
    load = 'data_' + str(i)
    data_saver_name.append(load) 
    load = web_scraper(url)
    new_df = pd.concat([new_df, load]).reset_index(drop=True)
    print(f'{100*((i+1)/num)}% of Data Loaded')
new_df = new_df[col]

Enter the number:2
Running Engine
This is the Page Title 
Nigeria 350 Jobs, 350 Job Vacancies in Nigeria - 350 Jobs in Nigeria | Hot Nigerian Jobs
Link Collection Successful 100%
Title Collection Successful 100%
Year Collection Successful 100%


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


50.0% of Data Loaded
Running Engine
This is the Page Title 
350 Jobs in Nigeria - Page 1 | Hot Nigerian Jobs
Link Collection Successful 100%
Title Collection Successful 100%
Year Collection Successful 100%
100.0% of Data Loaded


In [251]:
def data_saver(dataframe):
    name = input("Enter the name you want to save your file with: ")
    dataframe.to_csv(f'{name}.csv', index=False)
    print('Dataset has been saved to present working directory')

In [252]:
data_saver(data)

Enter the name you want to save your file withemmanuel
Dataset has been saved to present working directory


In [204]:
def description(data):
    desc = []
    for index, url in enumerate(data['Link']):
        yy = []
        cv = []
        page = requests.get(url).text
        soup = BeautifulSoup(page, features="lxml")
        des = soup.find("div", {"class":"mycase4"})

        for i in des.find_all('ul'):
            yy.append(str(i).replace('<li>', '** ').replace('</li>', '').replace('</ul>', '').replace('<ul>', ''))

        for i in des.find_all('p'):
            cv.append(str(i).replace('<p>', '** ').replace('</p>', '').replace('</strong>', '').replace('<strong>', '').replace('<br/>', ' '))
        yy.append(cv[0])
        
        desc.append(yy[0])
        print(index)
        
    return desc

In [209]:
dataframe = x.iloc[:8]

In [212]:
cc = description(dataframe)

0
1
2
3
4
5
6
7


In [215]:
dataframe['Job Information'] = cc

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [216]:
dataframe

,Title,Link,Year,Content,Location,Job Information
0,Admin and Finance Assistant at Mangrove and Pa...,https://www.hotnigerianjobs.com/hotjobs/323731...,0,2 - 6 years experience.,Abuja,\n** Have you seen anyone defecating in the op...
1,Accountant at Russell International School,https://www.hotnigerianjobs.com/hotjobs/323729...,2-6,Minimum 5 years experience,Lagos,\n** Preparing accounts and tax returns\n** Ad...
2,Project Accountants at Klinserv,https://www.hotnigerianjobs.com/hotjobs/323721...,5,1 - 3 years Experience.,Lagos,\n** BSc qualification in relevant discipline\...
3,Accounting Officers at Chartwell Securities Li...,https://www.hotnigerianjobs.com/hotjobs/323602...,0-1,"Candidates should possess Degree, HND, Master ...",Lagos,\n** Maintaining financial records.\n** Handli...
4,Account Officer at Ultimus Holdings,https://www.hotnigerianjobs.com/hotjobs/323595...,3-5,Experience: 3-5 years work experience,Lagos,\n** Monitor and control of the warehouses sal...
5,Accounting Intern at Ifgreen Industries & Inve...,https://www.hotnigerianjobs.com/hotjobs/323593...,0,Experience Length: No Experience / Less than 1...,Lagos,\n** We are looking for a driven Accounting In...
6,NYSC Internship Recruitment at Ifgreen Industr...,https://www.hotnigerianjobs.com/hotjobs/323591...,1,Develop and manage annual and multi-year budge...,Lagos,\n** Are you an NYSC intern looking for a plac...
7,Finance / Accounts Manager at SD Human Resourc...,https://www.hotnigerianjobs.com/hotjobs/323573...,0,Assist in preparing year-end books for audit,Lagos,\n** Develop and manage annual and multi-year ...
